In [20]:
# fordítás - működik
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

hu_text1 = "fiú"
hu_text2 = "férfi"

tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="hu")
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

encoded_hu1 = tokenizer(hu_text1, return_tensors="pt")
generated_tokens1 = model.generate(**encoded_hu1, forced_bos_token_id=tokenizer.get_lang_id("en"))
en_text1 = tokenizer.batch_decode(generated_tokens1, skip_special_tokens=True)[0]

encoded_hu2 = tokenizer(hu_text2, return_tensors="pt")
generated_tokens2 = model.generate(**encoded_hu2, forced_bos_token_id=tokenizer.get_lang_id("en"))
en_text2 = tokenizer.batch_decode(generated_tokens2, skip_special_tokens=True)[0]

print(hu_text1, en_text1,'\n',hu_text2, en_text2)

C:\Users\Miki\AppData\Roaming\Python\Python310\site-packages\transformers\generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


fiú boy 
 férfi Man


In [21]:
from transformers import M2M100Tokenizer
from transformers import M2M100Model
import torch

#m2m100_418M
#m2m100_1.2B better
model = M2M100Model.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="hu", tgt_lang="en")

Some weights of the model checkpoint at facebook/m2m100_418M were not used when initializing M2M100Model: ['lm_head.weight']
- This IS expected if you are initializing M2M100Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing M2M100Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of M2M100Model were not initialized from the model checkpoint at facebook/m2m100_418M and are newly initialized: ['model.encoder.embed_positions.weights', 'model.decoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# targettel működik
hu1 = "fiú"
#hu_target1="boy"
encoded_input1 = tokenizer(hu1, return_tensors='pt')
with tokenizer.as_target_tokenizer():
    labels1 = tokenizer(en_text1, return_tensors="pt").input_ids
output1=model(**encoded_input1, decoder_input_ids=labels1)
emb_hu1=output1[0][0][0]

hu2 = "férfi"
#hu_target2="man"
encoded_input2 = tokenizer(hu2, return_tensors='pt')
with tokenizer.as_target_tokenizer():
    labels2 = tokenizer(en_text2, return_tensors="pt").input_ids
output2 = model(**encoded_input2, decoder_input_ids=labels2)
emb_hu2=output2[0][0][0]

In [23]:
from scipy.spatial import distance

print(distance.cosine(emb_hu1.detach().numpy(),emb_hu2.detach().numpy())) #1.2B 0.0020917654037475586 fiu,ferfi  -  418M 0.0032167434692382812 fiu,ferfi

0.0032167434692382812


In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)